In [3]:
def get_posx_posy(xoffset, px_w, rot1, yoffset, px_h, rot2,x,y):
    # supposing x and y are your pixel coordinate this 
    # is how to get the coordinate in space.
    posX = px_w * x + rot1 * y + xoffset
    posY = rot2 * x + px_h * y + yoffset

    # shift to the center of the pixel
    posX += px_w / 2.0
    posY += px_h / 2.0
    return posX,posY

def get_poly_from_geotif_with_x_y(geotif_fp,minx,miny,maxx,maxy):
    ds = gdal.Open(geotif_fp)
    # open the dataset and get the geo transform matrix

    xoffset, px_w, rot1, yoffset, rot2,px_h = ds.GetGeoTransform()

    #print("xoffset, px_w, rot1, yoffset, px_h, rot2",xoffset, px_w, rot1, yoffset, px_h, rot2)
    print("minx,miny,maxx,maxy",minx,miny,maxx,maxy)

    pos1x,pos1y = get_posx_posy(xoffset, px_w, rot1, yoffset, px_h, rot2,minx,miny)
    pos2x,pos2y = get_posx_posy(xoffset, px_w, rot1, yoffset, px_h, rot2,minx,maxy)
    pos3x,pos3y = get_posx_posy(xoffset, px_w, rot1, yoffset, px_h, rot2,maxx,maxy)
    pos4x,pos4y = get_posx_posy(xoffset, px_w, rot1, yoffset, px_h, rot2,maxx,miny)
    coords = [(pos1x,pos1y), (pos2x,pos2y), (pos3x,pos3y), (pos4x,pos4y)]

    #print("pos",pos1x,pos1y,pos2x,pos2y,pos3x,pos3y,pos4x,pos4y)
    poly = Polygon(coords)
    
    return poly 

In [6]:
construction_type = "charcoal_hearth_hill"
cfg_name = 'cfg20200628T1152'
model_epoch='0016'


annot_prediction_folder = '/storage/images/'+construction_type+'/predictions/'+cfg_name+"/unknown/"
split_tifs_folder = '/storage/images/charcoal_hearth_hill/images/split_tifs/'
# display image with masks and bounding boxes
from os import listdir


from xml.etree import ElementTree
#from mrcnn.utils import Dataset
#from mrcnn.visualize import display_instances
#from mrcnn.utils import extract_bboxes
#https://gis.stackexchange.com/questions/92207/split-a-large-geotiff-into-smaller-regions-with-python-and-gdal

import numpy
from osgeo import gdal, osr
import math
from itertools import chain
import geopandas as gpd
from shapely.geometry import Point, Polygon
import numpy as np
import gdalnumeric

def put_preds_in_shp(state_area_num,state_area_num_crs):

    pred_polys = gpd.GeoDataFrame()
    pred_polys['geometry'] = None
    pred_polys.crs = {'init':'epsg:'+str(state_area_num_crs)}
    #pred_polys.crs = {'init':'epsg:32128'}

    import cv2

    #Store the results in XML    
    class_names = construction_type

    # find all images
    for annot_filename in listdir(annot_prediction_folder):
        print(annot_filename)
        #process only the files for this state land area, since other areas may not match crs
        if annot_filename.startswith(state_area_num):
            tree = ElementTree.parse(annot_prediction_folder+annot_filename)
            print(annot_prediction_folder+annot_filename)
            #print(tree)
            # get the root of the document
            root = tree.getroot()
            # extract each bounding box
    
            fn_image = root.find('./filename').text
            #object_present = root.find('./object_present').text
            fn_base = fn_image[:6]
            print(fn_base)
            box_num=0
            for obj in root.findall('./object'):
                score = obj.find('score').text
    
                box = obj.find('bndbox')
                box_num=obj.find('number').text
                box_num_pad = "00"+str(box_num)
                box_num_pad = box_num_pad[-2:]
                #boxes_correct[str(box_num)] = correct
                xmin = int(box.find('xmin').text)
                ymin = int(box.find('ymin').text)
                xmax = int(box.find('xmax').text)
                ymax = int(box.find('ymax').text)
                if(ymin>ymax):
                    ytemp = ymin
                    ymin = ymax
                    ymax=ytemp
                if(xmin>xmax):
                    xtemp = xmin
                    xmin = xmax
                    xmax=xtemp            
                coors = [xmin, ymin, xmax, ymax]
                print(coors)
        
                pred_poly = get_poly_from_geotif_with_x_y(split_tifs_folder+fn_base+".tif",xmin,ymin,xmax,ymax)
                new_pp_row = {'id':fn_base+box_num_pad, 'geometry':pred_poly}
                pred_polys = pred_polys.append(new_pp_row, ignore_index=True)

#pred_polys = pred_polys.to_crs(epsg=32128)
    outfp = "/storage/images/charcoal_hearth_hill/polys/"+cfg_name+"_"+state_area_num+"_predictions.shp"
# Write the data into that Shapefile
    pred_polys.to_file(outfp)
    pred_polys.head()
    pred_polys = pred_polys.to_crs({'init':'epsg:4326'})
    pred_polys.head()
    outfp = "/storage/images/charcoal_hearth_hill/polys/4326_"+cfg_name+"_"+state_area_num+"_predictions.shp"
    # Write the data into that Shapefile
    pred_polys.to_file(outfp)

 

In [ ]:
put_preds_in_shp("39",32128)
put_preds_in_shp("41",32129)
put_preds_in_shp("43",32129)
put_preds_in_shp("46",32129)
put_preds_in_shp("52",26918)
put_preds_in_shp("55",32128)
put_preds_in_shp("58",32128)

In [35]:

pred_polys = pred_polys.to_crs({'init':'epsg:4326'})
outfp = "/storage/images/charcoal_hearth_hill/polys/4326_"+cfg_name+"_predictions.shp"
# Write the data into that Shapefile
pred_polys.to_file(outfp)




In [31]:
pred_polys.head()

,geometry,id
0,"POLYGON ((546942.500 103703.500, 546942.500 10...",41121000
1,"POLYGON ((423114.500 132457.500, 423114.500 13...",39181400
2,"POLYGON ((422976.500 132482.500, 422976.500 13...",39181401
3,"POLYGON ((725111.500 98899.500, 725111.500 988...",46020500
4,"POLYGON ((724449.500 98985.500, 724449.500 989...",46020501
